In [ ]:
import Bio
from Bio import SeqIO
from Bio import Phylo
from Bio.Phylo.TreeConstruction import DistanceMatrix
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
import matplotlib
import matplotlib.pyplot as plt

# Read the database and mystery sequence
database = {record.id: str(record.seq) for record in SeqIO.parse("dog_breeds.fa", "fasta")}
unknown = {record.id: str(record.seq) for record in SeqIO.parse("mystery.fa", "fasta")}


def hamming_distance(seq1, seq2):
    """
    Determine hamming distance between two sequences i.e., number of mutations between two sequences
    """
    return sum([1 for x, y in zip(seq1, seq2) if x != y])


def top_n_similar_sequences(n, pairs):
    """
    Calculate the top N most similar sequences
    """
    sorted_pairs = sorted(pairs.items(), key=lambda x: x[1])
    top_n = sorted_pairs[:n]
    return top_n


# Take the mystery sequence from unknown dict
mystery = unknown['gb|KM061522.1|']
pairs = {}

# Iterate over all database sequences and compare with mystery sequence
for Id, seq in database.items():
    pairs[(Id, seq)] = hamming_distance(seq, mystery)

# Find out the closest breed and print the result
closest_breed = sorted(pairs.items(), key=lambda x: x[1])[0]
Id, seq, mut = closest_breed[0][0], closest_breed[0][1], closest_breed[1]

# Find the top 10 most similar sequences
top_10 = top_n_similar_sequences(10, pairs)

# Print the top 10 most similar sequences
print("Top 10 most similar sequences:")
print("Sequence ID\t% Similarity")

for item in top_10:
    id, seq = item[0]
    similarity = 100 - (item[1] / len(unknown['gb|KM061522.1|']) * 100)
    if id == Id:
        print(f"\033[1m{id}\t\t{similarity:.2f}%\033[0m\t#closest relative")
    else:
        print(f"{id}\t\t{similarity:.2f}%")

# Construct distance matrix to draw phylogenetic tree with Bio.Phylo module
ids = list(database.keys()) + list(unknown.keys())
sequences = list(database.values()) + list(unknown.values())

distM = [[0] * i for i in range(1, len(sequences) + 1)]
for i, x in enumerate(sequences):
    for j, y in zip(range(i), sequences):
        if i != j:
            distM[i][j] = hamming_distance(x, y)

# Construct and draw the phylogenetic tree with UPGMA method
dm = DistanceMatrix(ids, distM)
constructor = DistanceTreeConstructor()
tree = constructor.upgma(dm)
tree.root_at_midpoint()
tree.ladderize(reverse=True)
tree.root.color = 'gray'

# Create a figure and set size
fig = plt.figure(figsize=(10, 22), dpi=100)

# Set font size
matplotlib.rc('font', size=7)
axes = fig.add_subplot(1, 1, 1)
matplotlib.rc('xtick', labelsize=17)
matplotlib.rc('ytick', labelsize=17)
plt.xlabel('xlabel', fontsize=17)
plt.ylabel('ylabel', fontsize=17)

# Draw the phylogenetic tree
Phylo.draw(tree, axes=axes)

# Save the phylogenetic tree as an image file
plt.savefig("phylogenetic_tree.png")

# Show the phylogenetic tree
plt.show()


# Test functions
def test_hamming_distance():
    assert hamming_distance("ACTG", "ACTG") == 0
    assert hamming_distance("ACCG", "ACTG") == 1
    assert hamming_distance("AACG", "ACTG") == 2
    assert hamming_distance("AAAA", "ACTG") == 3
    assert hamming_distance("GGGG", "ACTG") == 3

def test_top_n_similar_sequences():
    pairs_test = {("id1", "seq1"): 1, ("id2", "seq2"): 5, ("id3", "seq3"): 3, ("id4", "seq4"): 2}
    result = top_n_similar_sequences(3, pairs_test)
    expected_result = [(("id1", "seq1"), 1), (("id4", "seq4"), 2), (("id3", "seq3"), 3)]
    assert result == expected_result


# Run tests
test_hamming_distance()
test_top_n_similar_sequences()
          

In [ ]:
#Install the rpy2 package by running this command in your terminal or Jupyter notebook:
!pip install rpy2
%load_ext rpy2.ipython
%%R
install.packages(c("ape", "tidytree", "ggtree"))
%%R -i dm -i Id
library(ape)
library(tidytree)
library(ggtree)

# Convert the Python distance matrix (dm) into an R distance matrix
dist_matrix <- as.dist(dm)

# Compute the tree using UPGMA method
tree <- upgma(dist_matrix)

# Convert the tree to a tidy format
tidy_tree <- as_tibble(tree)

# Plot the tree using ggtree
ggtree(tidy_tree) +
  geom_tiplab() +
  geom_point2(aes(subset = (node == Id), color = node, size = 5)) +
  scale_color_manual(values = "red", breaks = Id) +
  theme_tree2()